# Keras ile Deep Learning Yaklaşımı

- Bu Modelde ise diğerlerine oranla Churn Olmayanların Sayısı Churn olanların 2 katı olarak belirlendi, Burada ise veri setindeki kategorik değişkenler işlendikten sonra, Veriseti (-1,1) arasında scale edilip, Array haline getiriliyor ve model'in parametreleri verildikten sonra model ile eğitiliyor ardından test veriseti verilip tahminler alınıyor ve bu tahminler gerçek verisetiyle karşılaştırıldıktan sonra model'in ne kadar doğru çalıştığı ortaya çıkıyor.

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.noise import GaussianNoise
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils
import types
import tempfile
import keras.models
import keras
import pickle

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_excel("newdataFullv3.xlsx")
data = data.dropna()
columns = ['CEK_TUTAR',
 'VADE_GUN',
 'SON5ISTIHBARAT_SONUC',
 'SON5POLITIKA_SONUC',
 'SON5CEK_RENK_ORTALAMASI',
 'SON5KULLANDIRIM',
 'ŞUBE',
 'SIRKET_TURU',
 'MUSTERI_RISK_SEVIYESI',
 'TK_KURUMSAYISISON5',
 'TK_GECIKMEHESAPSON5',
 'TK_GECIKMEBAKIYEORT',
 'TK_NAKDILIMIT',
 'TK_NAKDIRISK',
 'TK_GAYRINAKDILIMIT',
 'BK_KURUMSAYISI',
 'BK_LIMIT',
 'BK_RISK',
 'BK_GECIKMEHESAP',
 'BK_GECIKMEBAKIYE',
 'T2_NAKDILIMIT_TPL',
 'T2_NAKDIRISK_TPL',
 'T2_NAKDIRISK_KV',
 'T2_GAYRINAKDILIMIT',
 'T2_GAYRINAKDIRISK',
 'T2_FKTRNG_LIMIT',
 'T2_FKTRNG_TPLRISK',
 'T2_NAKDIRISK_OV',
 'T2_NAKDIRISK_UV',
 'T2_SORUNLUKOD_RISK',
 'T3_NAKDILIMIT_TPL',
 'T3_NAKDIRISK_TPL',
 'T3_NAKDIRISK_KV',
 'T3_NAKDIRISK_OV',
 'T3_NAKDIRISK_UV',
 'T3_GAYRINAKDILIMIT',
 'T3_GAYRINAKDIRISK',
 'T3_FKTRNG_LIMIT',
 'T3_SORUNLUKOD_RISK',
 'T3_FKTRNG_TPLRISK',
 'T4_NAKDILIMIT_TPL',
 'T4_NAKDIRISK_TPL',
 'T4_NAKDIRISK_KV',
 'T4_NAKDIRISK_OV',
 'T4_NAKDIRISK_UV',
 'T4_GAYRINAKDILIMIT',
 'T4_GAYRINAKDIRISK',
 'T4_FKTRNG_LIMIT',
 'T4_SORUNLUKOD_RISK',
 'T4_FKTRNG_TPLRISK',
 'Churn']

data.head()

,CEK_TUTAR,VADE_GUN,SON5ISTIHBARAT_SONUC,SON5POLITIKA_SONUC,SON5CEK_RENK_ORTALAMASI,SON5KULLANDIRIM,ŞUBE,SIRKET_TURU,MUSTERI_RISK_SEVIYESI,TK_KURUMSAYISISON5,...,T4_NAKDIRISK_TPL,T4_NAKDIRISK_KV,T4_NAKDIRISK_OV,T4_NAKDIRISK_UV,T4_GAYRINAKDILIMIT,T4_GAYRINAKDIRISK,T4_FKTRNG_LIMIT,T4_SORUNLUKOD_RISK,T4_FKTRNG_TPLRISK,Churn
480,17126.636923,86.769231,RED,HAYR,Yesil,KY,IKITELLI,G,0,2,...,2951.2,2148.8,0.0,802.4,0.0,0.0,190000.0,0.0,54430.8,1
680,13715.801250,108.000000,ONAY,EVET,Yesil,KY,MECIDIYEKÖY,G,0,6,...,193811.2,161693.8,0.0,32117.4,44550.0,14400.0,390000.0,0.0,142473.2,1
1967,32722.478958,104.885417,RED,HAYR,Yesil,KY,IZMIT,T,0,5,...,181699.0,0.0,0.0,181699.0,14050.0,11200.0,5690000.0,0.0,496898.0,1
2161,56454.517292,159.562500,RED,HAYR,Siyah,KY,IKITELLI,T,2,14,...,6098286.0,2700782.8,1117431.4,2280071.8,4484608.0,3464409.0,10039565.8,0.0,3619009.6,1
2485,26900.000000,104.200000,RED,HAYR,Yesil,KY,TOPÇULAR,G,0,0,...,0.0,0.0,0.0,0.0,56000.0,0.0,846434.0,9137.0,159051.0,1


- Bu kısımda veri tiplerine göz atarak hangi verileri nasıl kullanacağıma ve kategorik verileri belirleyerek onları makine öğrenmesine uygun hale getirmek için veri tiplerine baktım

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19989 entries, 480 to SC00190565
Data columns (total 51 columns):
CEK_TUTAR                  19989 non-null float64
VADE_GUN                   19989 non-null float64
SON5ISTIHBARAT_SONUC       19989 non-null object
SON5POLITIKA_SONUC         19989 non-null object
SON5CEK_RENK_ORTALAMASI    19989 non-null object
SON5KULLANDIRIM            19989 non-null object
ŞUBE                       19989 non-null object
SIRKET_TURU                19989 non-null object
MUSTERI_RISK_SEVIYESI      19989 non-null int64
TK_KURUMSAYISISON5         19989 non-null int64
TK_GECIKMEHESAPSON5        19989 non-null int64
TK_GECIKMEBAKIYEORT        19989 non-null float64
TK_NAKDILIMIT              19989 non-null float64
TK_NAKDIRISK               19989 non-null float64
TK_GAYRINAKDILIMIT         19989 non-null float64
BK_KURUMSAYISI             19989 non-null int64
BK_LIMIT                   19989 non-null float64
BK_RISK                    19989 non-null float64
BK_

- Bu kısımda kategorik değişkenler numerik hale geliyor. Bunun içinde LabelEncoder adlı kütüphaneyi kullandım

In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

objects = ["SON5ISTIHBARAT_SONUC","SON5POLITIKA_SONUC","SON5CEK_RENK_ORTALAMASI","SON5KULLANDIRIM","ŞUBE","SIRKET_TURU"]
for i in objects:  
    data[i] = le.fit_transform(data[i])

- Bu kısımda ise elimde **1614 Churn** ve **18.375 Churn Olmayan** Data olduğu için overfitting yaşamamak adına, bunları ilk önce eşitledim, testlerim sonucunda ise Churn Sayısının 2 Katına kadar Churn olmayan alabileceğimi farkettim ve En optimize değer için bunu 2 kat olarak belirledim

- Not: Normalde diğer modeller bu oran 3 olsada keras'ta bunu 3 yapınca overfitting oluşuyordu bu yüzden bunu 2 ile sınırlandırdım.

In [5]:
dataChurn = data[data.Churn == 1]
dataNormal = data[data.Churn == 0]
data = pd.concat([dataNormal.sample(len(dataChurn)*3, random_state=39,replace=True), dataChurn])

- Burada ise veriyi Test ve Eğitim olmak üzere 2'ye böldüm verinin %20 si test %80'i train olarak ayrıldı.
- Aynı zamanda burada ayırdığım veri setlerini Array'a çevirerek (-1,1) Arasında Scale edip o şekilde modele yolladım.

In [6]:
X = data.iloc[:, data.columns != 'Churn']
y = data.Churn

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 25)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((-1,1))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

- Bu kısımda ise Model'in parametrelerini belirliyoruz. Burada:

- Eğitim Turu Sayısı (**Epoch**) : 1800
- Parça Sayısı (**Batch Size**): 200 Olarak Belirlendi. 
- Ardından Modele giriş layer'ını ekledik ve 2 adet hidden layer ekledik bunlardaki node sayısı ise **15**'ti Aktivasyon fonksiyonu olarak daha önce churn araştırmalarında daha doğrusu binary classification(0-1 sınıflandırması) araştırmalarında hard_sigmoid kullanıldığı için bunu tercih ettik. En son olarakta Çıktı layer'ımız verildi.

In [9]:
batch_size = 200
nb_epoch = 1800

model = Sequential()

model.add(Dense(55, input_shape=(50,)))
model.add(Activation("hard_sigmoid"))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(15))
model.add(Activation("hard_sigmoid"))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(15))
model.add(Activation("hard_sigmoid"))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam')

model.fit(X_train_scaled, y_train, batch_size=batch_size, epochs=nb_epoch, verbose=0)


- Bu kısımda ise eğittimiz modele test verisetini yollayarak tahmin değerlerini aldık, aldığımız tahmin değerleri 0-1 arasında olduğundan dolayı bunu 0.5'ten yukarısı 1 aşağısı 0 şeklinde kestik. Ve Tahminlerimizi gerçek sonuçlar ile karşılaştırdık.

In [10]:
y_pred = model.predict(X_test_scaled)
y_pred = (y_pred > 0.5) * 1

cm = confusion_matrix(y_test, y_pred)
print("confusion_matrix:\n", cm)
print("accuracy_score: ", accuracy_score(y_test, y_pred))

confusion_matrix:
 [[715 272]
 [ 98 207]]
accuracy_score:  0.7136222910216719


- Burada ise Modeli daha sonra Majority Vote Algoritmasında çağırabilmek için Ağırlıkları ve Mimarisi olarak 2 ayrı dosyada kaydettik.

In [8]:
#model.save_weights('model_weights.h5')

#with open('model_architecture.json', 'w') as f:
#    f.write(model.to_json())